# Understanding Hired Rides in NYC

_[Project prompt](https://docs.google.com/document/d/1uAUJGEUzfNj6OsWNAimnYCw7eKaHhMUfU1MTj9YwYw4/edit?usp=sharing), [grading rubric](https://docs.google.com/document/d/1hKuRWqFcIdhOkow3Nljcm7PXzIkoa9c_aHkMKZDxWa0/edit?usp=sharing)_

_This scaffolding notebook may be used to help setup your final project. It's **totally optional** whether you make use of this or not._

_If you do use this notebook, everything provided is optional as well - you may remove or add prose and code as you wish._

_**All code below should be consider "pseudo-code" - not functional by itself, and only an outline to help you with your own approach.**_

## Project Setup

In [31]:
pip install beautifulsoup4


Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install matplotlib


Note: you may need to restart the kernel to use updated packages.


In [33]:
pip install pandas

Note: you may need to restart the kernel to use updated packages.


In [34]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [35]:
pip install sqlalchemy

Note: you may need to restart the kernel to use updated packages.


In [36]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [38]:
# all import statements needed for the project, for example:

import math
import os
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
import pandas as pd
import requests
import sqlalchemy as db
import geopandas as gpd

In [39]:
# any constants you might need; some have been added for you, and 
# some you need to fill in

TAXI_URL = "https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page"

TAXI_ZONES_DIR = "data/taxi_zones"
TAXI_ZONES_SHAPEFILE = f"{TAXI_ZONES_DIR}/taxi_zones.shp"
UBER_CSV = ""
WEATHER_CSV_DIR = ""

CRS = 4326  # coordinate reference system

# (lat, lon)
NEW_YORK_BOX_COORDS = ((40.560445, -74.242330), (40.908524, -73.717047))
LGA_BOX_COORDS = ((40.763589, -73.891745), (40.778865, -73.854838))
JFK_BOX_COORDS = ((40.639263, -73.795642), (40.651376, -73.766264))
EWR_BOX_COORDS = ((40.686794, -74.194028), (40.699680, -74.165205))

DATABASE_URL = "sqlite:///project.db"
DATABASE_SCHEMA_FILE = "schema.sql"
QUERY_DIRECTORY = "queries"

In [40]:
# Make sure the QUERY_DIRECTORY exists
try:
    os.mkdir(QUERY_DIRECTORY)
except Exception as e:
    if e.errno == 17:
        # the directory already exists
        pass
    else:
        raise

## Part 1: Data Preprocessing

Load Taxi Zones

In [1]:
#def load_taxi_zones(shapefile):
   # if not os.path.isfile(shapefile):
    #    raise ValueError(f"Shapefile {shapefile} does not exist.")
   # gdf = gpd.read_file(shapefile)
   # gdf.crs = f"EPSG:{CRS}"
   # return gdf.to_crs("EPSG:4326")

def load_taxi_zones():
    shapefile_path = "https://github.com/Rayan3201/ProjectToolsForAnalytics/blob/3194c900e0730dc0930d1498d0c476411c4a8bf4/taxi_zones.shp"
    taxi_zones = gpd.read_file(shapefile_path)
    taxi_zones = taxi_zones[['LocationID', 'geometry']]
    return taxi_zones

def get_lat_lon(location_id, taxi_zones):
    zone = taxi_zones[taxi_zones['LocationID'] == location_id]
    if not zone.empty:
        lat, lon = zone.geometry.centroid.y.values[0], zone.geometry.centroid.x.values[0]
        return lat, lon
    else:
        return None, None

In [42]:
def lookup_coords_for_taxi_zone_id(zone_loc_id, loaded_taxi_zones):
    zone = loaded_taxi_zones[loaded_taxi_zones["LocationID"] == zone_loc_id]
    if len(zone) == 0:
        raise ValueError(f"Zone with LocationID {zone_loc_id} not found.")
    lon, lat = zone.geometry.centroid.iloc[0].coords[0]
    return lat, lon

### Calculate distance

In [ ]:
from geopy.distance import distance 
def calculate_distance_with_coords(from_coord, to_coord):
    return distance(from_coord, to_coord).miles

In [ ]:
def calculate_distance_with_zones(from_zone, to_zone):
    from_lat, from_lon = lookup_coords_for_taxi_zone_id(from_zone, taxi_zones_dict)
    to_lat, to_lon = lookup_coords_for_taxi_zone_id(to_zone, taxi_zones_dict)
    if from_lat is None or to_lat is None:
        return None
    from_point = (from_lat, from_lon)
    to_point = (to_lat, to_lon)
    distance = geopy.distance.distance(from_point, to_point).km
    return distance

In [ ]:
def calculate_distance(from_coord, to_coord):
    R = 6373.0

    lat1 = radians(from_coord[1])
    lon1 = radians(from_coord[0])
    lat2 = radians(to_coord[1])
    lon2 = radians(to_coord[0])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance
""" 
def add_distance_column(dataframe):
    dataframe['distance'] = dataframe.apply(
        lambda x: calculate_distance([x['pickup_latitude'],x['pickup_longitude']],
                                     [x['dropoff_latitude'],x['dropoff_longitude']]),
        axis =1
    )
"""

def add_distance_column(dataframe, taxi_zones):
    dataframe['pickup_lat_lon'] = dataframe['PULocationID'].apply(lambda x: get_lat_lon(x, taxi_zones))
    dataframe['dropoff_lat_lon'] = dataframe['DOLocationID'].apply(lambda x: get_lat_lon(x, taxi_zones))
    
    dataframe['distance'] = dataframe.apply(
        lambda x: calculate_distance(x['pickup_lat_lon'], x['dropoff_lat_lon']), axis=1)

### Process Taxi Data

In [ ]:
First Half 

In [43]:
import requests
import os
from bs4 import BeautifulSoup

def get_all_urls_from_taxi_page(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    
    links = soup.find_all('a')

    # Filter out the links that are not for parquet files
    parquet_links = []
    for link in links:
        href = link.get('href')
        if href and href.endswith('.parquet'):
            parquet_links.append(href)

    # Return the list of parquet file URLs
    return parquet_links

In [44]:
url = 'https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page'
parquet_links = get_all_urls_from_taxi_page(url)

# Print the list of parquet file URLs
print(parquet_links)

In [46]:
def filter_taxi_parquet_urls(parquet_links):
    # Filter out the parquet file URLs for yellow taxi data only
    yellow_taxi_links = []
    for link in parquet_links:
        if 'yellow_tripdata' in link:
            yellow_taxi_links.append(link)
    return yellow_taxi_links

In [ ]:
url = 'https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page'
parquet_links = get_all_urls_from_taxi_page(url)

In [ ]:
yellow_taxi_links = filter_taxi_parquet_urls(parquet_links)

In [ ]:
def download_yellow_taxi_parquet_files(parquet_links, directory):
    # Filter out the parquet file URLs for yellow taxi data only, from Jan 2009 to June 2015
    yellow_taxi_links = []
    for link in parquet_links:
        if 'yellow_tripdata' in link:
            match = re.search(r'yellow_tripdata_(\d{4})-(\d{2}).*', link)
            year, month = int(match.group(1)), int(match.group(2))
            if (year == 2009 and month >= 1) or (year >= 2010 and year <= 2014) or (year == 2015 and month <= 6):
                yellow_taxi_links.append(link)

    # Download yellow taxi parquet files to the specified directory
    for url in yellow_taxi_links:
        filename = url.split('/')[-1]
        response = requests.get(url, stream=True)
        with open(os.path.join(directory, filename), 'wb') as f:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    f.write(chunk)
    
    # Return the list of yellow taxi links
    return yellow_taxi_links

In [ ]:
import os

directory = '/Users/mikaelaballon/project_final/YELLOW PARQUET First half'

# Call the function to obtain the list of yellow taxi links
yellow_taxi_links = download_yellow_taxi_parquet_files(parquet_links, directory)

# Loop over the filtered list of yellow taxi parquet file URLs and download each file
for url in yellow_taxi_links:
    download_yellow_taxi_parquet_files(url, directory)

In [ ]:
def get_and_clean_month(url):
    # Define boundaries of New York City
    nyc_box = (-74.242330, -73.717047, 40.560445, 40.908524)

    # read the parquet file
    dataframe = pd.read_parquet(url)

    # Removing unnecessary columns and only keeping columns needed to answer questions in the other parts of this project
    columns_to_keep = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 'VendorID', 'passenger_count', 'trip_distance', 'PULocationID', 'DOLocationID', 'fare_amount', 'tip_amount']
    dataframe = dataframe[columns_to_keep]

    #normalize the column name
    dataframe.rename(columns = {"tip_amount":"Tip_Amt","tpep_pickup_datetime":"pickup_datetime", 'tpep_dropoff_datetime':'dropoff_datetime'}, inplace = True)

    # Remove values that are missing or invalid coordinates
    lat_long_cols = ['PULocationID', 'DOLocationID']
    for col in lat_long_cols:
        if f'{col}_latitude' in dataframe.columns and f'{col}_longitude' in dataframe.columns:
            invalid_coords = ((dataframe[f'{col}_latitude'] == 0) & (dataframe[f'{col}_longitude'] == 0))
            dataframe = dataframe[~invalid_coords]

    # Check if lat/long columns exist before attempting to extract them
    if all([f'{col}_longitude' in dataframe.columns for col in lat_long_cols]):
        # Remove trips outside the NYC boundaries
        pickup_longitude = dataframe[f'{lat_long_cols[0]}_longitude']
        pickup_latitude = dataframe[f'{lat_long_cols[0]}_latitude']
        dropoff_longitude = dataframe[f'{lat_long_cols[1]}_longitude']
        dropoff_latitude = dataframe[f'{lat_long_cols[1]}_latitude']
        outside_nyc = (
            (pickup_longitude < nyc_box[0]) |
            (pickup_longitude > nyc_box[1]) |
            (pickup_latitude < nyc_box[2]) |
            (pickup_latitude > nyc_box[3]) |
            (dropoff_longitude < nyc_box[0]) |
            (dropoff_longitude > nyc_box[1]) |
            (dropoff_latitude < nyc_box[2]) |
            (dropoff_latitude > nyc_box[3])
        )
        dataframe = dataframe[~outside_nyc]

    # Extracting date and hour from pickup_datetime column
    dataframe['pickup_date'] = dataframe['pickup_datetime'].dt.date
    dataframe['pickup_hour'] = dataframe['pickup_datetime'].dt.hour

    # New column for trip duration in minutes
    dataframe['trip_duration_minutes'] = (dataframe['dropoff_datetime'] - dataframe['pickup_datetime']).dt.seconds/60

    # Dropping the original pickup_datetime and dropoff_datetime columns
    dataframe.drop(['pickup_datetime', 'dropoff_datetime'], axis=1, inplace=True)

    return dataframe

In [ ]:
"""
def get_and_clean_taxi_data(parquet_urls):
    all_taxi_dataframes = []
    
    for parquet_url in parquet_urls:
        # maybe: first try to see if you've downloaded this exact
        # file already and saved it before trying again
        dataframe = get_and_clean_month(parquet_url)
        add_distance_column(dataframe)
        # maybe: if the file hasn't been saved, save it so you can
        # avoid re-downloading it if you re-run the function
        
        all_taxi_dataframes.append(dataframe)
        
    # create one gigantic dataframe with data from every month needed
    taxi_data = pd.contact(all_taxi_dataframes)
    return taxi_data

""" 

def get_and_clean_taxi_data(parquet_urls):
    taxi_zones = load_taxi_zones()  # Load taxi zones
    taxi_data = pd.DataFrame()
    for parquet_url in parquet_urls:
        dataframe = get_and_clean_month(parquet_url)
        add_distance_column(dataframe, taxi_zones)  # Pass taxi_zones here
        taxi_data = pd.concat([taxi_data, dataframe], ignore_index=True)
    return taxi_data

In [53]:
def get_taxi_data():
    all_urls = get_all_urls_from_taxi_page(TAXI_URL)
    all_parquet_urls = filter_taxi_parquet_urls(all_urls, dir_path)
    taxi_data = get_and_clean_taxi_data(all_parquet_urls)
    return taxi_data

In [54]:
taxi_data = get_taxi_data()

/Users/mikaelaballon/project_final/yello_taxis/yellow_tripdata_2023-01.parquet has been downloaded.
/Users/mikaelaballon/project_final/yello_taxis/yellow_tripdata_2022-01.parquet has been downloaded.
/Users/mikaelaballon/project_final/yello_taxis/yellow_tripdata_2022-02.parquet has been downloaded.
/Users/mikaelaballon/project_final/yello_taxis/yellow_tripdata_2022-03.parquet has been downloaded.
/Users/mikaelaballon/project_final/yello_taxis/yellow_tripdata_2022-04.parquet has been downloaded.
/Users/mikaelaballon/project_final/yello_taxis/yellow_tripdata_2022-05.parquet has been downloaded.
/Users/mikaelaballon/project_final/yello_taxis/yellow_tripdata_2022-06.parquet has been downloaded.
/Users/mikaelaballon/project_final/yello_taxis/yellow_tripdata_2022-07.parquet has been downloaded.
/Users/mikaelaballon/project_final/yello_taxis/yellow_tripdata_2022-08.parquet has been downloaded.
/Users/mikaelaballon/project_final/yello_taxis/yellow_tripdata_2022-09.parquet has been downloaded.


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x8b in position 19: invalid start byte

In [ ]:
taxi_data.head()

: 

### Processing Uber Data

In [ ]:
import pandas as pd
def load_and_clean_uber_data(csv_file):
    dataframe = pd.read_csv(r'C:/Users/mikaelaballon/project_final/uber/uber_rides_sample.csv')

In [55]:
import pandas as pd
UBER_DATA = "/Users/mikaelaballon/project_final/uber/uber_rides_sample.csv"

def load_and_clean_uber_data(UBER_DATA):
    dataframe = pd.read_csv(UBER_DATA)

    dataframe = dataframe.drop(['Unnamed: 0','key'], axis=1)
    dataframe = dataframe.dropna(subset=['pickup_latitude', 'pickup_longitude', 'dropoff_latitude','dropoff_longitude'])

    min_lat, min_lon = 40.560445, -74.242330
    max_lat, max_lon = 40.908524, -73.717047

    dataframe = dataframe[(dataframe['pickup_latitude'].between(min_lat, max_lat)) &
                          (dataframe['pickup_longitude'].between(min_lon, max_lon)) &
                          (dataframe['dropoff_latitude'].between(min_lat, max_lat)) &
                          (dataframe['dropoff_longitude'].between(min_lon, max_lon))]
    dataframe['pickup_datetime'] = pd.to_datetime(dataframe['pickup_datetime'])
    
    return dataframe

In [56]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [57]:
from geopy.distance import geodesic

In [58]:
from geopy.distance import geodesic

def add_distance_column(uber_dataframe):
    pickup_coords = list(zip(uber_dataframe['pickup_latitude'], uber_dataframe['pickup_longitude']))
    dropoff_coords = list(zip(uber_dataframe['dropoff_latitude'], uber_dataframe['dropoff_longitude']))
    distances = [geodesic(pickup, dropoff).km for pickup, dropoff in zip(pickup_coords, dropoff_coords)]
    uber_dataframe['distance_km'] = distances

In [59]:
def get_uber_data():
    uber_dataframe = load_and_clean_uber_data(UBER_DATA)
    add_distance_column(uber_dataframe)
    return uber_dataframe

In [60]:
uber_data = get_uber_data()

In [ ]:
uber_data.head()

: 

### Processing Weather Data

In [ ]:
directory = '/Users/mikaelaballon/project_final/weather data'

def get_all_weather_csvs(directory):
    csv_files = []
    for filename in os.listdir(directory):
        if filename.endswith('.csv'):
            csv_files.append(os.path.join(directory, filename))
    return csv_files

: 

In [ ]:
def clean_month_weather_data_hourly(csv_file):
    # Load the CSV file into a Pandas dataframe
    dataframe = pd.read_csv(csv_file)

    # Convert 'date_time' column to datetime data type
    dataframe['date_time'] = pd.to_datetime(dataframe['date_time'])

    # Set 'date_time' column as the index
    dataframe.set_index('date_time', inplace=True)

    # Resample the data to hourly frequency and interpolate missing values
    dataframe = dataframe.resample('H').mean().interpolate()

    # Removing unnecessary columns and only keeping columns needed to answer questions in the other parts of this project
    columns_to_keep = ['pickup_datetime', 'dropoff_datetime', 'pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude', 'fare_amount', 'passenger_count']
    dataframe = dataframe.loc[:, columns_to_keep]

    # Remove values that are missing or invalid coordinates
    dataframe.dropna(subset=['pickup_latitude', 'pickup_longitude', 'dropoff_latitude', 'dropoff_longitude'], inplace=True)
    invalid_coords = ((dataframe['pickup_latitude'] == 0) & (dataframe['pickup_longitude'] == 0) & (dataframe['dropoff_latitude'] == 0) & (dataframe['dropoff_longitude'] == 0))
    dataframe = dataframe[~invalid_coords]

    # Extracting date and hour from pickup_datetime column
    dataframe['pickup_date'] = dataframe.index.date
    dataframe['pickup_hour'] = dataframe.index.hour

    # New column for trip duration in minutes
    dataframe['trip_duration_minutes'] = (dataframe['dropoff_datetime'] - dataframe['pickup_datetime']).dt.total_seconds() / 60

    # Ensure all trips have positive times and are not over two hours
    dataframe = dataframe[(dataframe['trip_duration_minutes'] > 0) & (dataframe['trip_duration_minutes'] < 120)]

    # Remove trips with no fare
    dataframe = dataframe[dataframe['fare_amount'] > 0]

    # Define boundaries of New York City
    nyc_box = (-74.242330, -73.717047, 40.560445, 40.908524)

    # Remove trips that are outside the New York boundaries
    dataframe = dataframe[(dataframe['pickup_longitude'] >= nyc_box[0]) & (dataframe['pickup_longitude'] <= nyc_box[1]) &
    (dataframe['pickup_latitude'] >= nyc_box[2]) & (dataframe['pickup_latitude'] <= nyc_box[3]) &
    (dataframe['dropoff_longitude'] >= nyc_box[0]) & (dataframe['dropoff_longitude'] <= nyc_box[1]) &
    (dataframe['dropoff_latitude'] >= nyc_box[2]) & (dataframe['dropoff_latitude'] <= nyc_box[3])]
    
    return dataframe

: 

In [ ]:
def clean_month_weather_data_daily(csv_file):
    # Load the CSV file into a pandas dataframe
    dataframe = pd.read_csv(csv_file)

    # Removing unnecessary columns and only keeping columns needed to answer questions in the other parts of this project
    columns_to_keep = ['pickup_datetime', 'fare_amount', 'passenger_count']
    dataframe = dataframe.loc[:, columns_to_keep]

    # Converting datetime column to datetime data type
    dataframe['pickup_datetime'] = pd.to_datetime(dataframe['pickup_datetime'])

    # Extracting date from pickup_datetime column
    dataframe['pickup_date'] = dataframe['pickup_datetime'].dt.date

    # Dropping the original pickup_datetime column
    dataframe.drop(['pickup_datetime'], axis=1, inplace=True)

    # Remove trips with no fare
    dataframe = dataframe[dataframe['fare_amount'] > 0]

    # Define boundaries of New York City
    nyc_box = (-74.242330, -73.717047, 40.560445, 40.908524)

    # Remove trips that are outside the New York boundaries
    dataframe = dataframe[(dataframe['pickup_longitude'] >= nyc_box[0]) & (dataframe['pickup_longitude'] <= nyc_box[1]) &
    (dataframe['pickup_latitude'] >= nyc_box[2]) & (dataframe['pickup_latitude'] <= nyc_box[3])]

    # Group data by date and calculate the total fare and passenger count for each date
    daily_data = dataframe.groupby(['pickup_date']).agg({'fare_amount': 'sum', 'passenger_count': 'sum'}).reset_index()

    return daily_data

: 

In [ ]:
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(WEATHER_CSV_DIR)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

: 

In [ ]:
def load_and_clean_weather_data():
    weather_csv_files = get_all_weather_csvs(directory)
    
    hourly_dataframes = []
    daily_dataframes = []
        
    for csv_file in weather_csv_files:
        hourly_dataframe = clean_month_weather_data_hourly(csv_file)
        daily_dataframe = clean_month_weather_data_daily(csv_file)
        hourly_dataframes.append(hourly_dataframe)
        daily_dataframes.append(daily_dataframe)
        
    # create two dataframes with hourly & daily data from every month
    hourly_data = pd.concat(hourly_dataframes)
    daily_data = pd.concat(daily_dataframes)
    
    return hourly_data, daily_data

In [ ]:
hourly_weather_data, daily_weather_data = load_and_clean_weather_data()

: 

In [ ]:
hourly_weather_data.head()

: 

In [ ]:
daily_weather_data.head()

: 

## Part 2: Storing Cleaned Data

In [3]:
engine = db.create_engine(DATABASE_URL)

NameError: name 'db' is not defined

In [ ]:
# if using SQL (as opposed to SQLAlchemy), define the commands 
# to create your 4 tables/dataframes
HOURLY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS hourly_weather (
    id INTEGER PRIMARY KEY,
    datetime TEXT,
    temperature REAL,
    precipitation REAL,
    wind_speed REAL,
    visibility REAL
);
"""


DAILY_WEATHER_SCHEMA = """
CREATE TABLE IF NOT EXISTS daily_weather (
    id INTEGER PRIMARY KEY,
    date TEXT,
    max_temperature REAL,
    min_temperature REAL,
    precipitation REAL,
    wind_speed REAL,
    visibility REAL
);
"""

TAXI_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS taxi_trips (
    id INTEGER PRIMARY KEY,
    pickup_datetime TEXT,
    dropoff_datetime TEXT,
    passenger_count INTEGER,
    trip_distance REAL,
    pickup_latitude REAL,
    pickup_longitude REAL,
    dropoff_latitude REAL,
    dropoff_longitude REAL,
    distance REAL
);
"""

UBER_TRIPS_SCHEMA = """
CREATE TABLE IF NOT EXISTS uber_trips (
    id INTEGER PRIMARY KEY,
    pickup_datetime TEXT,
    dropoff_datetime TEXT,
    passenger_count INTEGER,
    trip_distance REAL,
    pickup_latitude REAL,
    pickup_longitude REAL,
    dropoff_latitude REAL,
    dropoff_longitude REAL,
    distance REAL
);
"""

: 

In [ ]:
# create that required schema.sql file
with open(DATABASE_SCHEMA_FILE, "w") as f:
    f.write(HOURLY_WEATHER_SCHEMA)
    f.write(DAILY_WEATHER_SCHEMA)
    f.write(TAXI_TRIPS_SCHEMA)
    f.write(UBER_TRIPS_SCHEMA)

: 

In [ ]:
# create the tables with the schema files
with engine.connect() as connection:
    connection.execute(HOURLY_WEATHER_SCHEMA)
    connection.execute(DAILY_WEATHER_SCHEMA)
    connection.execute(TAXI_TRIPS_SCHEMA)
    connection.execute(UBER_TRIPS_SCHEMA)

: 

### Add Data to Database

In [ ]:
def write_dataframes_to_table(table_to_df_dict):
    for table_name, df in table_to_df_dict.items():
        df.to_sql(table_name, engine, if_exists='replace', index=False)

: 

In [ ]:
map_table_name_to_dataframe = {
    "taxi_trips": taxi_data,
    "uber_trips": uber_data,
    "hourly_weather": hourly_data,
    "daily_weather": daily_data,
}

: 

In [ ]:
write_dataframes_to_table(map_table_name_to_dataframe)

: 

## Part 3: Understanding the Data

In [ ]:
# Helper function to write the queries to file
def write_query_to_file(query, outfile):
    raise NotImplementedError()

: 

### Query 1

In [ ]:
QUERY_1_FILENAME = "q1.sq1"

QUERY_1 = """
WITH hourly_pickups AS (
    SELECT strftime('%H', pickup_datetime) AS hour, COUNT(*) AS num_rides
    FROM taxi_trips
    WHERE strftime('%Y-%m', pickup_datetime) BETWEEN '2009-01' AND '2015-06'
    GROUP BY hour
)
SELECT hour, num_rides
FROM hourly_pickups
ORDER BY num_rides DESC;
"""

QUERY_2_FILENAME = "q2.sq1"

QUERY_2 = """
WITH daily_pickups AS (
    SELECT strftime('%w', pickup_datetime) AS day_of_week, COUNT(*) AS num_rides
    FROM uber_trips
    WHERE strftime('%Y-%m', pickup_datetime) BETWEEN '2009-01' AND '2015-06'
    GROUP BY day_of_week
)
SELECT day_of_week, num_rides
FROM daily_pickups
ORDER BY num_rides DESC;
"""


QUERY_3_FILENAME = "q3.sq1"

QUERY_3 = """
WITH july_trips AS (
    SELECT distance
    FROM taxi_trips
    WHERE strftime('%Y-%m', pickup_datetime) = '2013-07'
)
SELECT percentile_cont(0.95) WITHIN GROUP (ORDER BY distance) AS percentile_95
FROM july_trips;
"""

QUERY_4_FILENAME = "q4.sq1"

QUERY_4 = """
WITH daily_summary AS (
    SELECT strftime('%Y-%m-%d', pickup_datetime) AS date,
           COUNT(*) AS num_rides,
           AVG(distance) AS avg_distance
    FROM taxi_trips
    WHERE strftime('%Y', pickup_datetime) = '2009'
    GROUP BY date
)
SELECT date, num_rides, avg_distance
FROM daily_summary
ORDER BY num_rides DESC
LIMIT 10;
"""

QUERY_5_FILENAME = "q5.sq1"

QUERY_5 = """
WITH windiest_days AS (
    SELECT date, AVG(wind_speed) AS avg_wind_speed
    FROM daily_weather
    WHERE strftime('%Y', date) = '2014'
    GROUP BY date
    ORDER BY avg_wind_speed DESC
    LIMIT 10
),
trips_on_windiest_days AS (
    SELECT windiest_days.date,
           windiest_days.avg_wind_speed,
           COUNT(taxi_trips.id) AS num_trips
    FROM windiest_days
    LEFT JOIN taxi_trips ON strftime('%Y-%m-%d', taxi_trips.pickup_datetime) = windiest_days.date
    GROUP BY windiest_days.date, windiest_days.avg_wind_speed
)
SELECT date, avg_wind_speed, num_trips
FROM trips_on_windiest_days
ORDER BY avg
"""

QUERY_6_FILENAME = "q6.sq1"

QUERY_6 = """

"""

: 

In [ ]:
engine.execute(QUERY_1).fetchall()
engine.execute(QUERY_2).fetchall()
engine.execute(QUERY_3).fetchall()
engine.execute(QUERY_4).fetchall()
engine.execute(QUERY_5).fetchall()
engine.execute(QUERY_6).fetchall()

: 

In [ ]:
write_query_to_file(QUERY_1, QUERY_1_FILENAME)
write_query_to_file(QUERY_2, QUERY_1_FILENAME)
write_query_to_file(QUERY_3, QUERY_1_FILENAME)
write_query_to_file(QUERY_4, QUERY_1_FILENAME)
write_query_to_file(QUERY_5, QUERY_1_FILENAME)
write_query_to_file(QUERY_6, QUERY_1_FILENAME)

: 

## Part 4: Visualizing the Data 

### Visualization 1

In [ ]:
def plot_visual_1(dataframe):
    plt.figure(figsize=(10, 6))
    sns.barplot(x='borough', y='average_fare', data=dataframe)
    plt.title('Average Fare per Borough')
    plt.xlabel('Borough')
    plt.ylabel('Average Fare')
    plt.show()

def get_data_for_visual_1():
    conn = sqlite3.connect(DATABASE_URL)
    query = '''SELECT ... FROM ...'''
    dataframe = pd.read_sql_query(QUERY_1, conn)
    conn.close()
    return dataframe

some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)

: 

In [ ]:
def plot_visual_2(dataframe):
    plt.figure(figsize=(10, 6))
    sns.lineplot(x='month', y='average_distance', data=dataframe)
    plt.title('Average Distance Traveled per Month')
    plt.xlabel('Month')
    plt.ylabel('Average Distance')
    plt.show()

def get_data_for_visual_2():
    conn = sqlite3.connect(DATABASE_URL)
    query = '''
        SELECT strftime('%m', date_column) AS month, AVG(distance_column) AS average_distance
        FROM uber_trips, taxi_trips
        GROUP BY month
    '''
    dataframe = pd.read_sql_query(QUERY_2, conn)
    conn.close()
    return dataframe

some_dataframe = get_data_for_visual_2()
plot_visual_2(some_dataframe)

: 

In [ ]:
def plot_visual_3(dataframe):
    plt.figure(figsize=(10, 6))
    sns.barplot(x='airport', y='dropoff_count', hue='day_of_week', data=dataframe)
    plt.title('Drop-offs at Airports by Day of the Week')
    plt.xlabel('Airport')
    plt.ylabel('Drop-off Count')
    plt.show()

def get_data_for_visual_3():
    conn = sqlite3.connect(DATABASE_URL)
    query = '''
        SELECT airport, strftime('%w', date_column) AS day_of_week, COUNT(*) AS dropoff_count
        FROM uber_trips, taxi_trips
        WHERE dropoff_location IN ('LGA', 'JFK', 'EWR')
        GROUP BY airport, day_of_week
    '''
    dataframe = pd.read_sql_query(QUERY_3, conn)
    conn.close()
    return dataframe

some_dataframe = get_data_for_visual_3()
plot_visual_3(some_dataframe)

In [ ]:
import folium
from folium.plugins import HeatMap

def plot_visual_4(dataframe):
    m = folium.Map([40.7128, -74.0060], zoom_start=11)
    
    heatmap_data = dataframe[['latitude', 'longitude']].values.tolist()
    
    HeatMap(heatmap_data).add_to(m)
    
    return m

def get_data_for_visual_4():
    conn = sqlite3.connect(DATABASE_URL)
    query = '''
        SELECT latitude, longitude
        FROM uber_trips, taxi_trips
    '''
    dataframe = pd.read_sql_query(QUERY_4, conn)
    conn.close()
    return dataframe

some_dataframe = get_data_for_visual_4()
plot_visual_4(some_dataframe)

In [ ]:
def plot_visual_5(dataframe):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='distance', y='tip_amount', data=dataframe)
    plt.title('Tip Amount vs Distance for Yellow Taxi Rides')
    plt.xlabel('Distance')
    plt.ylabel('Tip Amount')
    plt.show()

def get_data_for_visual_5():
    conn = sqlite3.connect(DATABASE_URL)
    query = '''
        SELECT distance, tip_amount
        FROM taxi_trips
    '''
    dataframe = pd.read_sql_query(QUERY_5, conn)
    conn.close()
    return dataframe

some_dataframe = get_data_for_visual_5()
plot_visual_5(some_dataframe)

In [ ]:
def plot_visual_6(dataframe):
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x='precipitation_amount', y='tip_amount', data=dataframe)
    plt.title('Tip Amount vs Precipitation Amount for Yellow Taxi Rides')
    plt.xlabel('Precipitation Amount (inches)')
    plt.ylabel('Tip Amount ($)')
    plt.show()

def get_data_for_visual_6():
    query = '''
    SELECT
        yellow_taxi.tip_amount,
        weather.precipitation
    FROM taxi_trips
    JOIN weather ON yellow_taxi.pickup_datetime = weather.date_time
    WHERE yellow_taxi.tip_amount > 0
    '''
    dataframe = pd.read_sql(QUERY_6, conn)
    conn.close()
    return dataframe

some_dataframe = get_data_for_visual_5()
plot_visual_5(some_dataframe)

In [ ]:
some_dataframe = get_data_for_visual_1()
plot_visual_1(some_dataframe)

: 